<a href="https://colab.research.google.com/github/nddoshi/pointnet/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%cd /content    # start in content
import os       # import os

# clone in git repo if it doesn't exist, update it if it does
if not os.path.isdir('/content/pointnet'):
  !git clone https://github.com/nddoshi/pointnet.git
else:
  %cd pointnet
  !git pull  

%cd /content/pointnet

/content/pointnet
Already up to date.
/content/pointnet


In [12]:
# copy in ModelNet10 data set
if not os.path.isdir('/content/datasets'):
  %mkdir /content/datasets
if not os.path.isdir('/content/datasets/ModelNet10'):
  %cd /content/datasets
  !wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
  !unzip -q ModelNet10.zip
  %rm -rf __MACOSX
  %rm ModelNet10.zip

%cd /content/pointnet

/content/pointnet


In [37]:
# imports 
import math
import random
import sys,os
import torch
import argparse
!pip install path
from path import Path
from source import model
from source import dataset
from source import utils
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np

random.seed = 42

1
Tesla K80


In [3]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha=0.0001):
    criterion = torch.nn.NLLLoss()
    bs = outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs, 1, 1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs, 1, 1)
    if outputs.is_cuda:
        id3x3 = id3x3.cuda()
        id64x64 = id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3, m3x3.transpose(1, 2))
    diff64x64 = id64x64-torch.bmm(m64x64, m64x64.transpose(1, 2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)


In [ ]:
# arguments

root_dir = '../datasets/ModelNet10'
batch_size=32
lr =1e-3
epochs=15
save_model_path = './checkpoints/'


path = Path(root_dir)
folders = [dir for dir in sorted(
    os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)}

train_transforms = transforms.Compose([
    utils.PointSampler(1024),
    utils.Normalize(),
    utils.RandRotation_z(),
    utils.RandomNoise(),
    utils.ToTensor()
])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
pointnet = model.PointNet()
pointnet.to(device)
optimizer = torch.optim.Adam(pointnet.parameters(), lr=lr)

train_ds = dataset.PointCloudData(path, transform=train_transforms)
valid_ds = dataset.PointCloudData(
    path, valid=True, folder='test', transform=train_transforms)
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))

train_loader = DataLoader(
    dataset=train_ds, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=batch_size*2)

try:
    os.mkdir(save_model_path)
except OSError as error:
    print(error)

print('Start training')
for epoch in range(epochs):
  pointnet.train()
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
      inputs, labels = data['pointcloud'].to(
          device).float(), data['category'].to(device)
      optimizer.zero_grad()
      outputs, m3x3, m64x64 = pointnet(inputs.transpose(1, 2))

      loss = pointnetloss(outputs, labels, m3x3, m64x64)
      loss.backward()
      optimizer.step()

      # print statistics
      running_loss += loss.item()
      if i % 10 == 9:    # print every 10 mini-batches
          print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                (epoch + 1, i + 1, len(train_loader), running_loss / 10))
          running_loss = 0.0
    
  pointnet.eval()
  correct = total = 0

  # validation
  if valid_loader:
      with torch.no_grad():
          for data in valid_loader:
              inputs, labels = data['pointcloud'].to(
                  device).float(), data['category'].to(device)
              outputs, __, __ = pointnet(inputs.transpose(1, 2))
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
      val_acc = 100. * correct / total
      print('Valid accuracy: %d %%' % val_acc)
  # save the model

  checkpoint = Path(save_model_path)/'save_'+str(epoch)+'.pth'
  torch.save(pointnet.state_dict(), checkpoint)
  print('Model saved to ', checkpoint)


cuda:0
Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
[Errno 17] File exists: './checkpoints/'
Start training
[Epoch: 1, Batch:   10 /  125], loss: 1.968
[Epoch: 1, Batch:   20 /  125], loss: 1.592
[Epoch: 1, Batch:   30 /  125], loss: 1.498
[Epoch: 1, Batch:   40 /  125], loss: 1.292
[Epoch: 1, Batch:   50 /  125], loss: 1.153
[Epoch: 1, Batch:   60 /  125], loss: 1.158
[Epoch: 1, Batch:   70 /  125], loss: 1.053
[Epoch: 1, Batch:   80 /  125], loss: 1.016
[Epoch: 1, Batch:   90 /  125], loss: 0.975
